In [7]:
import csv
import re

def generate_insert_statements(
    csv_file, 
    create_stream_statement, 
    output_file, 
    table_name,
    encoding='utf-8'
):
    # Extract content within parentheses
    stream_content = re.search(r'\((.*?)\)', create_stream_statement, re.DOTALL).group(1)
    
    # Split content into individual column definitions
    columns = re.findall(r'([a-zA-Z_]+)\s+([a-zA-Z_]+)', stream_content)

    # Extract column names and data types, stripping BOM if present
    column_names_create_stream = [col[0].strip() for col in columns]
    data_types_create_stream = {col[0].strip(): col[1] for col in columns}

    # Open the CSV file
    with open(csv_file, newline='', encoding=encoding, errors='ignore') as csvfile:
        reader = csv.DictReader(csvfile)
        insert_statements = []

        # Iterate over each row in the CSV
        for row in reader:
            columns = []
            values = []

            # Iterate over each column in the row
            for field, value in row.items():
                if field in column_names_create_stream:
                    # Column name exists in create_stream_statement
                    data_type = data_types_create_stream[field]
                    if value is None:
                        columns.append(field)
                        values.append("")
                    else:
                        if data_type == "DOUBLE" or data_type == "INTEGER":
                            value_str = str(value)
                            if value_str.strip() == "":
                                columns.append(field)
                                values.append("")
                            else:
                                columns.append(field)
                                values.append(value_str)
                        elif data_type == "TIMESTAMP" or data_type == "DATE":
                            columns.append(field)
                            values.append("'" + str(value) + "'")
                        elif data_type == "VARCHAR":
                            columns.append(field)
                            values.append("'" + value.replace("'", "''") + "'")
                        # Handle other data types as needed

            # Construct the INSERT INTO statement with the specified table name
            insert_statement = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(values)});"
            insert_statements.append(insert_statement)

    # Write insert statements to output file
    with open(output_file, 'w', encoding=encoding, errors='ignore') as f:
        for statement in insert_statements:
            f.write(statement + '\n')

In [9]:
create_stream_statement = """
CREATE STREAM car_specs (
    id INTEGER,
    model VARCHAR,
    serie VARCHAR,
    company VARCHAR,
    body_style VARCHAR,
    segment VARCHAR,
    production_years ARRAY[INTEGER],
    cylinders VARCHAR,
    displacement VARCHAR,
    power_hp VARCHAR,
    power_bhp VARCHAR,
    power_kw VARCHAR,
    torque_lb-ft VARCHAR,
    torque_nm VARCHAR,
    electrical_motor_power VARCHAR,
    electrical_motor_torque VARCHAR,
    fuel_system VARCHAR,
    fuel VARCHAR,
    fuel_capacity VARCHAR,
    top_speed VARCHAR,
    acceleration_0_62 VARCHAR,
    drive_type VARCHAR,
    gearbox VARCHAR,
    front brake VARCHAR,
    rear brake VARCHAR,
    tire_size VARCHAR,
    car_length VARCHAR,
    car_width VARCHAR,
    car_height VARCHAR,
    front_rear_track VARCHAR,
    wheelbase VARCHAR,
    ground_clearance VARCHAR,
    aerodynamics_cd VARCHAR,
    aerodynamics_frontal_area VARCHAR,
    turning_circle VARCHAR,
    cargo_volume VARCHAR,
    unladen_weight VARCHAR,
    gross_weight_limit VARCHAR,
    combined_mpg VARCHAR,
    city_mpg VARCHAR,
    highway_mpg VARCHAR,
    co2_emissions VARCHAR,
    co2_emissions_combined VARCHAR,
    turning_circle_curb_to_curb VARCHAR,
    total_max_power VARCHAR,
    power_pack VARCHAR,
    nominal_capacity VARCHAR,
    top_speed_electrical VARCHAR,
    ev_range VARCHAR,
    high_mpg VARCHAR,
    extra_high_mpg VARCHAR,
    medium_mpg VARCHAR,
    low_mpg VARCHAR,
    total_max_torque VARCHAR,
    max_capacity VARCHAR,
    spec_summary VARCHAR
)
"""

input_data_file = '/Users/briandunn/Desktop/Kafka/ksql/Car Specification Dataset/data/cars-dataset.csv'

output_file_location = '/Users/briandunn/Desktop/Kafka/ksql/Car Specification Dataset/statements/producer/car_spec_inserts.sql'

generate_insert_statements(
    input_data_file, 
    create_stream_statement, 
    output_file_location,
    "cars_specs"
    )